In [2]:
buytest=read.csv("buytest.csv", header=T)
str(buytest)
dim(buytest)

'data.frame':	10000 obs. of  26 variables:
 $ ID      : Factor w/ 10000 levels "000054889","000219612",..: 11 18 25 101 140 150 162 178 187 209 ...
 $ RESPOND : int  0 0 0 0 0 0 0 0 0 0 ...
 $ AGE     : int  71 53 53 45 32 35 43 39 66 NA ...
 $ INCOME  : int  67 72 70 56 66 48 49 64 65 NA ...
 $ SEX     : Factor w/ 3 levels "","F","M": 3 3 2 2 2 2 2 3 3 1 ...
 $ MARRIED : int  1 1 1 0 0 0 0 0 0 NA ...
 $ FICO    : int  719 751 725 684 651 691 694 659 692 707 ...
 $ OWNHOME : int  0 0 0 0 0 1 1 0 0 NA ...
 $ LOC     : Factor w/ 8 levels "A","B","C","D",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ CLIMATE : int  10 10 10 10 10 10 10 10 10 10 ...
 $ BUY6    : int  1 0 1 0 0 0 0 0 0 0 ...
 $ BUY12   : int  1 0 1 0 0 0 0 0 0 0 ...
 $ BUY18   : int  1 0 1 1 0 0 0 0 0 0 ...
 $ VALUE24 : int  318 83 265 448 161 250 194 446 214 198 ...
 $ ORGSRC  : Factor w/ 8 levels "","C","D","I",..: 5 7 3 5 7 2 7 3 5 5 ...
 $ DISCBUY : int  1 0 0 1 0 0 0 0 1 0 ...
 $ RETURN24: int  0 0 0 0 0 1 1 0 0 0 ...
 $ COA6    : int

[1] 10000    26

In [4]:
buytest$RESPOND=as.factor(buytest$RESPOND)
buytest$MARRIED=as.factor(buytest$MARRIED)
buytest$COA6=as.factor(buytest$COA6)
buytest$OWNHOME=as.factor(buytest$OWNHOME)
buytest$CLIMATE=as.factor(buytest$CLIMATE)
buytest$DISCBUY=as.factor(buytest$DISCBUY)

In [5]:
buytest1=buytest[,1:18]

일부 변수만 선택

RESPOND를 타겟으로 로지스틱 회귀분석

## split data 

In [6]:
set.seed(1234) 
ind <- sample(2, nrow(buytest1), replace=T, prob=c(0.7, 0.3))
trainData <- buytest1[ind==1,]
testData <- buytest1[ind==2,]

## model 

In [7]:
out_r=glm(RESPOND~AGE+CLIMATE, family=binomial, data=trainData)
summary(out_r)


Call:
glm(formula = RESPOND ~ AGE + CLIMATE, family = binomial, data = trainData)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-0.6500  -0.4276  -0.3869  -0.3473   2.5725  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept) -0.961133   0.213809  -4.495 6.95e-06 ***
AGE         -0.027007   0.004512  -5.986 2.15e-09 ***
CLIMATE20   -0.412689   0.112614  -3.665 0.000248 ***
CLIMATE30   -0.393119   0.140619  -2.796 0.005180 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 3783.4  on 6870  degrees of freedom
Residual deviance: 3734.0  on 6867  degrees of freedom
  (170 observations deleted due to missingness)
AIC: 3742

Number of Fisher Scoring iterations: 5


age와 climate만 독립변수로 respond를 예측하는데 이 때 climate는 범주형 변수로 팩터화가 되어있다. 그리하여 회귀모델에서는 자동으로 더미 변수로 변환된다. climate는 10, 20, 30 세 범주로 나뉘는데 세 범주이므로 더미 변수는 CLIMATE20, CLIMATE30 두 개이다.

In [8]:
names(out_r)

[1] "coefficients"      "residuals"         "fitted.values"    
 [4] "effects"           "R"                 "rank"             
 [7] "qr"                "family"            "linear.predictors"
[10] "deviance"          "aic"               "null.deviance"    
[13] "iter"              "weights"           "prior.weights"    
[16] "df.residual"       "df.null"           "y"                
[19] "converged"         "boundary"          "model"            
[22] "na.action"         "call"              "formula"          
[25] "terms"             "data"              "offset"           
[28] "control"           "method"            "contrasts"        
[31] "xlevels"

In [9]:
1/exp(out_r$coefficients)

(Intercept)         AGE   CLIMATE20   CLIMATE30 
   2.614656    1.027375    1.510875    1.481595

계수들이 음수였으므로 1/exp()를 하여 오즈비를 보았는데 예를 들어 나이 AGE가 1증가하면 respond일 확률이 1.027배씩 감소한다.

## modeling and selection

In [10]:
mine=RESPOND~AGE+INCOME+SEX+MARRIED+FICO+OWNHOME+LOC+CLIMATE+BUY6+BUY12+BUY18+VALUE24+
             ORGSRC+DISCBUY+RETURN24+COA6

In [11]:
out_f=glm(mine, family=binomial, data=trainData)
summary(out_f)


Call:
glm(formula = mine, family = binomial, data = trainData)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.1689  -0.4328  -0.3606  -0.2962   2.8740  

Coefficients: (2 not defined because of singularities)
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)  6.017e-01  1.148e+00   0.524   0.6002    
AGE         -3.787e-02  5.222e-03  -7.252 4.10e-13 ***
INCOME      -2.067e-04  2.960e-03  -0.070   0.9443    
SEXM         2.411e-03  9.369e-02   0.026   0.9795    
MARRIED1     5.369e-01  1.052e-01   5.102 3.36e-07 ***
FICO        -2.948e-03  1.553e-03  -1.898   0.0577 .  
OWNHOME1    -4.263e-01  1.085e-01  -3.929 8.53e-05 ***
LOCB        -1.202e-02  2.137e-01  -0.056   0.9551    
LOCC         3.848e-01  2.494e-01   1.543   0.1229    
LOCD         3.738e-01  2.483e-01   1.506   0.1322    
LOCE        -9.767e-02  2.148e-01  -0.455   0.6493    
LOCF        -1.089e-01  2.158e-01  -0.505   0.6137    
LOCG        -2.257e-01  2.504e-01  -0.901   0.3674 

In [12]:
out_step=step(out_f, direction="both")

Start:  AIC=3662.07
RESPOND ~ AGE + INCOME + SEX + MARRIED + FICO + OWNHOME + LOC + 
    CLIMATE + BUY6 + BUY12 + BUY18 + VALUE24 + ORGSRC + DISCBUY + 
    RETURN24 + COA6


Step:  AIC=3662.07
RESPOND ~ AGE + INCOME + SEX + MARRIED + FICO + OWNHOME + LOC + 
    BUY6 + BUY12 + BUY18 + VALUE24 + ORGSRC + DISCBUY + RETURN24 + 
    COA6

           Df Deviance    AIC
- ORGSRC    7   3613.6 3655.6
- BUY6      1   3606.1 3660.1
- SEX       1   3606.1 3660.1
- INCOME    1   3606.1 3660.1
- VALUE24   1   3606.1 3660.1
- RETURN24  1   3606.1 3660.1
- COA6      1   3606.3 3660.3
- DISCBUY   1   3606.3 3660.3
<none>          3606.1 3662.1
- LOC       7   3620.2 3662.2
- FICO      1   3609.7 3663.7
- BUY12     1   3614.7 3668.7
- OWNHOME   1   3622.3 3676.3
- MARRIED   1   3632.9 3686.9
- BUY18     1   3641.8 3695.8
- AGE       1   3660.8 3714.8

Step:  AIC=3655.58
RESPOND ~ AGE + INCOME + SEX + MARRIED + FICO + OWNHOME + LOC + 
    BUY6 + BUY12 + BUY18 + VALUE24 + DISCBUY + RETURN24 + COA6

     

In [13]:
summary(out_step)


Call:
glm(formula = RESPOND ~ AGE + MARRIED + FICO + OWNHOME + LOC + 
    BUY12 + BUY18, family = binomial, data = trainData)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.1925  -0.4311  -0.3635  -0.2997   2.8973  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)  0.781673   1.105404   0.707 0.479481    
AGE         -0.038215   0.005183  -7.373 1.67e-13 ***
MARRIED1     0.540174   0.104816   5.154 2.56e-07 ***
FICO        -0.002841   0.001544  -1.840 0.065745 .  
OWNHOME1    -0.424566   0.107417  -3.953 7.73e-05 ***
LOCB        -0.001011   0.213219  -0.005 0.996216    
LOCC         0.389431   0.248059   1.570 0.116435    
LOCD         0.379282   0.247372   1.533 0.125216    
LOCE        -0.085818   0.214119  -0.401 0.688571    
LOCF        -0.103036   0.215315  -0.479 0.632270    
LOCG        -0.214750   0.243710  -0.881 0.378224    
LOCH        -0.055722   0.232643  -0.240 0.810704    
BUY12       -0.528015   0.138913  -3.801 0

In [14]:
exp(out_step$coefficients)

(Intercept)         AGE    MARRIED1        FICO    OWNHOME1        LOCB 
  2.1851248   0.9625061   1.7163063   0.9971630   0.6540535   0.9989893 
       LOCC        LOCD        LOCE        LOCF        LOCG        LOCH 
  1.4761413   1.4612353   0.9177614   0.9020949   0.8067430   0.9458019 
      BUY12       BUY18 
  0.5897744   2.1130943

계수가 음수인 것들은 1보다 작음.

## validation

In [15]:
p_train=predict(out_step,trainData, type="response")
p_test=predict(out_step, newdata=testData, type="response")

In [18]:
table(trainData$RESPOND,p_train>0.5)
table(trainData$RESPOND,p_train>0.1)
table(testData$RESPOND,p_test>0.1)

   
    FALSE TRUE
  0  6303    1
  1   540    0

   
    FALSE TRUE
  0  5065 1239
  1   312  228

   
    FALSE TRUE
  0  2173  521
  1   120   69

threshold를 0.5로 했을 경우 거의 대부분이 FALSE로 나오므로 변화를 주어 0.5로 바꾸고 test에 적용한 교차표